In [1]:
import os
os.chdir("C:/Users/david/Desktop/Uni/potato-dry-matter-optics-ml")

from dotenv import load_dotenv

load_dotenv()  # lee el archivo .env

ROBOFLOW_API_KEY = os.environ["ROBOFLOW_API_KEY"]
os.environ["ROBOFLOW_API_KEY"] = ROBOFLOW_API_KEY

# Comprobación rápida
print("ROBOFLOW_API_KEY configurada?:", "ROBOFLOW_API_KEY" in os.environ)

ROBOFLOW_API_KEY configurada?: True


In [2]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key=ROBOFLOW_API_KEY
)

In [3]:
IMAGE_PATH = "data/input/raw/raw_images/test_0/test_img2.jpg"

result = CLIENT.infer(IMAGE_PATH, model_id="coco-dataset-vdnr1/23")

In [4]:
print(result)

{'inference_id': 'f05ec0e1-ccfe-45f1-9f44-dffdc880b458', 'time': 0.10029306099750102, 'image': {'width': 427, 'height': 443}, 'predictions': [{'x': 198.5, 'y': 212.0, 'width': 351.0, 'height': 400.0, 'confidence': 0.9853866100311279, 'class': 'orange', 'points': [{'x': 162.6640625, 'y': 12.459375381469727}, {'x': 161.9718780517578, 'y': 13.151562690734863}, {'x': 161.27969360351562, 'y': 13.151562690734863}, {'x': 160.58750915527344, 'y': 13.84375}, {'x': 159.8953094482422, 'y': 13.84375}, {'x': 159.203125, 'y': 14.535937309265137}, {'x': 158.5109405517578, 'y': 14.535937309265137}, {'x': 157.81875610351562, 'y': 15.22812557220459}, {'x': 157.12657165527344, 'y': 15.22812557220459}, {'x': 156.4343719482422, 'y': 15.920312881469727}, {'x': 154.35781860351562, 'y': 15.920312881469727}, {'x': 153.66563415527344, 'y': 16.61250114440918}, {'x': 152.28125, 'y': 16.61250114440918}, {'x': 151.5890655517578, 'y': 17.3046875}, {'x': 150.89688110351562, 'y': 17.3046875}, {'x': 150.20469665527344,

In [5]:
from PIL import Image, ImageDraw
import numpy as np
import os
from typing import List, Tuple, Optional, Dict, Any

In [6]:
def _prediction_to_points(pred: Dict[str, Any]) -> Optional[List[Tuple[float, float]]]:
    """
    Extrae puntos de contorno si existen en el JSON de Roboflow.
    """
    pts = pred.get("points") or pred.get("polygon") or pred.get("vertices") or pred.get("segmentation")
    if not pts:
        return None

    if isinstance(pts, list) and len(pts) > 0:
        if isinstance(pts[0], dict):
            out = []
            for p in pts:
                x = p.get("x")
                y = p.get("y")
                if x is not None and y is not None:
                    out.append((float(x), float(y)))
            return out if len(out) >= 3 else None

        if isinstance(pts[0], (list, tuple)) and len(pts[0]) >= 2:
            out = [(float(p[0]), float(p[1])) for p in pts]
            return out if len(out) >= 3 else None

    return None


def polygon_to_mask(points: List[Tuple[float, float]], w: int, h: int) -> np.ndarray:
    """
    Rasteriza un polígono a máscara binaria.
    """
    mask_img = Image.new("L", (w, h), 0)
    draw = ImageDraw.Draw(mask_img)
    pts_int = [(int(x), int(y)) for x, y in points]
    draw.polygon(pts_int, outline=1, fill=1)
    return np.array(mask_img, dtype=np.uint8)


def _normalize_mask(m: Any, w: int, h: int) -> Optional[np.ndarray]:
    """
    Convierte un campo 'mask' del JSON a np.ndarray (h, w) binario.
    """
    if m is None:
        return None

    if isinstance(m, np.ndarray):
        arr = m
    elif isinstance(m, (list, tuple)):
        arr = np.array(m)
    elif isinstance(m, dict) and "data" in m:
        # por si viene como {data: [...]}
        arr = np.array(m["data"])
    else:
        try:
            arr = np.array(m)
        except:
            return None

    # Asegurar 2D
    if arr.ndim == 3:
        # posibles formatos raros
        if arr.shape[-1] == 1:
            arr = arr[..., 0]
        elif arr.shape[0] == 1:
            arr = arr[0]

    if arr.ndim != 2:
        return None

    arr = (arr > 0).astype(np.uint8)

    # Ajuste de tamaño si hiciera falta
    if arr.shape != (h, w):
        if arr.size == h * w:
            arr = arr.reshape((h, w))
        else:
            mask_img = Image.fromarray(arr * 255)
            mask_img = mask_img.resize((w, h), resample=Image.NEAREST)
            arr = (np.array(mask_img) > 0).astype(np.uint8)

    return arr


def extract_instance_masks_from_result(result: Dict[str, Any], w: int, h: int) -> List[np.ndarray]:
    """
    Intenta extraer máscaras por instancia desde result.
    Prioriza:
      1) points -> máscara
      2) mask directo
    """
    masks = []
    preds = result.get("predictions", []) if isinstance(result, dict) else []

    # 1) Si hay polígonos
    for pred in preds:
        points = _prediction_to_points(pred)
        if points:
            masks.append(polygon_to_mask(points, w, h))

    if masks:
        return masks

    # 2) Si hay máscaras directas
    for pred in preds:
        nm = _normalize_mask(pred.get("mask") or pred.get("segmentation_mask"), w, h)
        if nm is not None:
            masks.append(nm)

    return masks


def mask_boundary(mask: np.ndarray) -> np.ndarray:
    """
    Calcula borde de una máscara binaria sin OpenCV.
    """
    m = mask.astype(bool)
    h, w = m.shape

    up    = np.zeros_like(m); up[1:, :] = m[:-1, :]
    down  = np.zeros_like(m); down[:-1, :] = m[1:, :]
    left  = np.zeros_like(m); left[:, 1:] = m[:, :-1]
    right = np.zeros_like(m); right[:, :-1] = m[:, 1:]

    interior = up & down & left & right & m
    boundary = m & (~interior)
    return boundary.astype(np.uint8)

In [10]:
from typing import Dict, Any, Optional, Tuple
from PIL import Image, ImageDraw
import numpy as np
import os


def erode_mask(mask: np.ndarray, margin: int) -> np.ndarray:
    """
    Erosiona una máscara binaria aplicando un margen hacia dentro.
    
    Parámetros
    ----------
    mask : np.ndarray
        Máscara binaria (h, w) con valores 0 o 1.
    margin : int
        Número de píxeles a erosionar desde el borde.
    
    Retorna
    -------
    np.ndarray
        Máscara erosionada.
    """
    if margin <= 0:
        return mask
    
    h, w = mask.shape
    eroded = mask.copy().astype(bool)
    
    # Aplicar erosión iterativamente
    for _ in range(margin):
        temp = eroded.copy()
        for i in range(h):
            for j in range(w):
                if eroded[i, j]:
                    # Verificar vecinos (4-conectividad)
                    neighbors = []
                    if i > 0: neighbors.append(temp[i-1, j])
                    if i < h-1: neighbors.append(temp[i+1, j])
                    if j > 0: neighbors.append(temp[i, j-1])
                    if j < w-1: neighbors.append(temp[i, j+1])
                    
                    # Si algún vecino es 0, este píxel se erosiona
                    if not all(neighbors):
                        eroded[i, j] = False
    
    return eroded.astype(np.uint8)


def get_mask_bounding_box(mask: np.ndarray) -> Optional[Tuple[int, int, int, int]]:
    """
    Calcula el bounding box de una máscara.
    
    Retorna
    -------
    tuple o None
        (x1, y1, x2, y2) del bounding box, o None si la máscara está vacía.
    """
    rows, cols = np.where(mask > 0)
    if len(rows) == 0:
        return None
    
    y1, y2 = int(rows.min()), int(rows.max()) + 1
    x1, x2 = int(cols.min()), int(cols.max()) + 1
    
    return (x1, y1, x2, y2)


def save_contour_plus_eroded_area(
    image_path: str,
    result: Dict[str, Any],
    output_path: str,
    margin: int = 0,
) -> Optional[Tuple[np.ndarray, Tuple[int, int, int, int]]]:
    """
    Guarda una imagen con:
      - el contorno combinado de todas las máscaras (en rojo),
      - el área erosionada por el margen (en verde).
    
    Parámetros
    ----------
    image_path : str
        Ruta a la imagen original.
    result : dict
        Resultado de la inferencia de Roboflow.
    output_path : str
        Ruta donde guardar la visualización.
    margin : int
        Número de píxeles a erosionar desde el borde.
    
    Retorna
    -------
    tuple o None
        (mask_erosionada, bbox) donde bbox es (x1, y1, x2, y2), o None si no hay máscaras.
    """
    image = Image.open(image_path).convert("RGB")
    w, h = image.size
    
    masks = extract_instance_masks_from_result(result, w, h)
    
    if not masks:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        image.save(output_path)
        return None
    
    # Combinar todas las máscaras
    combined_mask = np.zeros((h, w), dtype=np.uint8)
    for mask in masks:
        combined_mask = np.maximum(combined_mask, mask)
    
    # Calcular borde de la máscara original
    global_boundary = mask_boundary(combined_mask)
    
    # Erosionar la máscara combinada
    eroded_mask = erode_mask(combined_mask, margin)
    
    # Calcular borde de la máscara erosionada
    eroded_boundary = mask_boundary(eroded_mask)
    
    # Obtener bounding box del área erosionada
    bbox = get_mask_bounding_box(eroded_mask)
    
    # Visualización
    vis = image.copy()
    draw = ImageDraw.Draw(vis)
    
    # Contorno original en rojo
    if global_boundary.any():
        red = (255, 0, 0)
        ys, xs = np.where(global_boundary == 1)
        for x, y in zip(xs.tolist(), ys.tolist()):
            vis.putpixel((x, y), red)
    
    # Contorno erosionado en verde
    if eroded_boundary.any():
        green = (0, 255, 0)
        ys, xs = np.where(eroded_boundary == 1)
        for x, y in zip(xs.tolist(), ys.tolist()):
            vis.putpixel((x, y), green)
    
    # Bounding box en azul (opcional, para referencia)
    if bbox is not None:
        draw.rectangle(bbox, outline="blue", width=2)
    
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    vis.save(output_path)
    
    if bbox is None:
        return None
    
    return (eroded_mask, bbox)


def save_eroded_mask_crop(
    image_path: str,
    eroded_mask: np.ndarray,
    bbox: Tuple[int, int, int, int],
    output_path: str,
    apply_mask: bool = True,
) -> Tuple[int, int, int, int]:
    """
    Guarda un recorte de la imagen usando el área erosionada.
    
    Parámetros
    ----------
    image_path : str
        Ruta a la imagen original.
    eroded_mask : np.ndarray
        Máscara erosionada.
    bbox : tuple
        Bounding box (x1, y1, x2, y2) del área erosionada.
    output_path : str
        Ruta donde guardar el recorte.
    apply_mask : bool
        Si True, aplica la máscara (píxeles fuera = negro).
        Si False, solo recorta el bounding box.
    
    Retorna
    -------
    tuple
        Las coordenadas del bounding box usado: (x1, y1, x2, y2).
    """
    image = Image.open(image_path).convert("RGB")
    img_array = np.array(image)
    
    x1, y1, x2, y2 = bbox
    
    # Recortar la imagen y la máscara
    cropped_img = img_array[y1:y2, x1:x2].copy()
    cropped_mask = eroded_mask[y1:y2, x1:x2]
    
    if apply_mask:
        # Aplicar la máscara: píxeles fuera de la máscara = negro
        cropped_img[cropped_mask == 0] = 0
    
    # Guardar
    result_img = Image.fromarray(cropped_img)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    result_img.save(output_path)
    
    return bbox

In [11]:
OUTPUT_PATH = "data/input/processed_images/test_0/test_img2_visualize.jpg"
margin = 10

result_data = save_contour_plus_eroded_area(
    IMAGE_PATH,
    result,
    OUTPUT_PATH,
    margin=margin,
)

if result_data is not None:
    eroded_mask, bbox = result_data
    print("=== BOUNDING BOX DEL ÁREA EROSIONADA (x1, y1, x2, y2) ===")
    print(bbox)
    print("=== IMAGEN DE VISUALIZACIÓN GUARDADA ===")
    print(OUTPUT_PATH)
    
    # Guardar el recorte
    CROPPED_OUTPUT_PATH = "data/input/processed_images/test_0/test_img2_cut.jpg"
    
    final_bbox = save_eroded_mask_crop(
        IMAGE_PATH,
        eroded_mask,
        bbox,
        CROPPED_OUTPUT_PATH,
        apply_mask=True,  # True = aplica máscara, False = solo bbox
    )
    
    print("=== IMAGEN RECORTADA GUARDADA ===")
    print(CROPPED_OUTPUT_PATH)
    print("Bbox usado:", final_bbox)
else:
    print("No se encontraron máscaras en la imagen")

=== BOUNDING BOX DEL ÁREA EROSIONADA (x1, y1, x2, y2) ===
(32, 22, 363, 402)
=== IMAGEN DE VISUALIZACIÓN GUARDADA ===
data/input/processed_images/test_0/test_img2_visualize.jpg
=== IMAGEN RECORTADA GUARDADA ===
data/input/processed_images/test_0/test_img2_cut.jpg
Bbox usado: (32, 22, 363, 402)
